In [34]:
import pandas as pd

In [35]:
# Load the datasets
indian_food_df = pd.read_csv("indian_food.csv")
good_ingredients_mood_df = pd.read_csv("ingredients_mood-good.csv")
bad_ingredients_mood_df = pd.read_csv("ingredients_mood-bad.csv")

In [36]:
# Example to filter based on mood to get relevant ingredients
def get_dishes_based_on_mood(mood):
    # Make empty dataframes for positive and negative dishes
    positive_dishes = pd.DataFrame(columns=indian_food_df.columns)
    negative_dishes = pd.DataFrame(columns=indian_food_df.columns)

    # Get all ingredients that have a non NaN value in the respective mood column
    positive_ingredients = good_ingredients_mood_df[good_ingredients_mood_df[mood].notna()]
    negative_ingredients = bad_ingredients_mood_df[bad_ingredients_mood_df[mood].notna()]

    # positive_ingredients = positive_ingredients.iloc[:, 0].str.lower()
    # negative_ingredients = negative_ingredients.iloc[:, 0].str.lower()

    # Get dishes to recommend from the positive_ingredients df based on the mood
    for index, row in indian_food_df.iterrows():
        for ingredient in positive_ingredients["ingredients"]:
            if ingredient.lower() in row["ingredients"].lower():
                positive_dishes = pd.concat([positive_dishes, indian_food_df[indian_food_df["name"] == row["name"]]])
        for ingredient in negative_ingredients["ingredients"]:
            if ingredient.lower() in row["ingredients"].lower():
                negative_dishes = pd.concat([positive_dishes, indian_food_df[indian_food_df["name"] == row["name"]]])

    # Remove duplicate rows in both dataframes
    positive_dishes = positive_dishes.drop_duplicates()
    negative_dishes = negative_dishes.drop_duplicates()

    return positive_dishes, negative_dishes


In [37]:
get_dishes_based_on_mood("Disgust")[0]

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
11,Lassi,"Yogurt, milk, nuts, sugar",vegetarian,5,5,sweet,dessert,Punjab,North
38,Bandar laddu,"Besan, jaggery, cardamom powder, ghee, cashews...",vegetarian,5,35,sweet,dessert,Andhra Pradesh,South
64,Maach Jhol,"Fish, potol, tomato, chillies, ginger, garlic",non vegetarian,10,40,spicy,main course,Assam,North East
...,...,...,...,...,...,...,...,...,...
239,Koldil Chicken,"Banana flower, chicken, green chili, mustard o...",non vegetarian,-1,-1,spicy,main course,Assam,North East
242,Masor Koni,"Fish roe, pumpkin flowers, mustard oil, turmer...",non vegetarian,-1,-1,spicy,main course,Assam,North East
243,Mishti Chholar Dal,"Chana dal, fresh coconut, ginger, cinnamon, ra...",vegetarian,10,30,sweet,main course,West Bengal,East
247,Prawn malai curry,"Coconut milk, prawns, garlic, turmeric, sugar",non vegetarian,15,50,spicy,main course,West Bengal,East


In [38]:
def recommend_dishes(mood, allergies, is_vegan, like_spicy):
    # Get relevant ingredients for the mood
    positive_dishes, negative_dishes = get_dishes_based_on_mood(mood)
    positive_ingredients = good_ingredients_mood_df[good_ingredients_mood_df[mood].notna()]
    negative_ingredients = bad_ingredients_mood_df[bad_ingredients_mood_df[mood].notna()]

    # Exclude based on user restrictions
    if is_vegan:
        positive_dishes = positive_dishes[positive_dishes["diet"] == "vegetarian"]
    for allergen in allergies:
        positive_dishes = positive_dishes[~positive_dishes['ingredients'].str.contains(allergen, case=False)]
    if not like_spicy:
        positive_dishes = positive_dishes[~positive_dishes['flavor_profile'].str.contains("spicy", case=False)]

    final_recom = positive_dishes.sample(n=3)  # For simplicity, randomly pick three satisfying all conditions
    # final_recom = positive_dishes.head(3)

    # Check if final_recom is empty
    if final_recom.empty:
        print("Sorry, I couldn't find any dishes satisfying your preferences and restrictions.")
    else:
        print(f"Based on your mood and preferences, I recommend trying the following dishes: ")
        for index, row in final_recom.iterrows():
            print(f"{row['name']} from the {row['state']} state of the {row['region']} region, which is a {row['course']} dish.")
            print(f"{row['name']} is a {row['flavor_profile']} dish and is {row['diet']}.")
            print(f"{row['name']} contains the ingredients {row['ingredients']}.")
            for ingredient in row['ingredients'].split(","):
                # print(ingredient)
                ingredient = ingredient.strip()
                # print(positive_ingredients)
                # FIXME: Check if ingredient is really missing from positive_ingredients["ingredients"] or it's just a difference in capitalization
                # FIXME: If it's the former, then why is it missing if we got our recommendation based on the ingredients?
                if ingredient in positive_ingredients["ingredients"].values:
                    # print(positive_ingredients.loc[positive_ingredients['ingredients'] == ingredient, "Y Reason"].iloc[0])
                    ingredient_reason = positive_ingredients.loc[positive_ingredients['ingredients'] == ingredient, "Y Reason"].iloc[0]
                    # print(ingredient_reason)
                    if isinstance(ingredient_reason, str):
                        print("  " + ingredient_reason)

In [39]:
positive_ingredients2 = good_ingredients_mood_df[good_ingredients_mood_df["Disgust"].notna()]
positive_ingredients2

,ingredients,Sad,Anger,Disgust,Fear,Joy,Neutral,Y Reason,Source,translation
8,potato,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
10,garlic,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
11,ginger,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
21,fish,y,NaN,y,NaN,NaN,NaN,Fish contains Omega-3 and is a part of the Med...,NaN,NaN
23,Potatoes,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
24,hot water,NaN,NaN,y,NaN,NaN,NaN,"When feeling sick, hot water helps relieve con...",https://www.healthline.com/nutrition/drinking-...,NaN
40,yogurt,y,y,y,y,NaN,NaN,Yogurt is rich in Tryptophan. Tryptophan can i...,NaN,NaN
72,Chicken thighs,y,NaN,y,NaN,NaN,NaN,Chicken is a part of the Mediterranean diet.Th...,NaN,NaN
77,garlic powder,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
80,chicken,y,y,y,NaN,NaN,NaN,Chicken contains Tryptophan and Selenium and i...,NaN,NaN


In [41]:
isinstance(positive_ingredients2.loc[positive_ingredients2['ingredients'] == "fish", "Y Reason"].iloc[0], str)

True

In [42]:
indian_food_df.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


In [43]:
good_ingredients_mood_df.head()

,ingredients,Sad,Anger,Disgust,Fear,Joy,Neutral,Y Reason,Source,translation
0,elachi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardamom
1,vegetable oil,y,NaN,NaN,NaN,NaN,NaN,Vegetable oil contains vitamin E which has bee...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,NaN
2,ghee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,jaggery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
4,Rice flour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
get_dishes_based_on_mood("Disgust")[0]

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
11,Lassi,"Yogurt, milk, nuts, sugar",vegetarian,5,5,sweet,dessert,Punjab,North
38,Bandar laddu,"Besan, jaggery, cardamom powder, ghee, cashews...",vegetarian,5,35,sweet,dessert,Andhra Pradesh,South
64,Maach Jhol,"Fish, potol, tomato, chillies, ginger, garlic",non vegetarian,10,40,spicy,main course,Assam,North East
...,...,...,...,...,...,...,...,...,...
239,Koldil Chicken,"Banana flower, chicken, green chili, mustard o...",non vegetarian,-1,-1,spicy,main course,Assam,North East
242,Masor Koni,"Fish roe, pumpkin flowers, mustard oil, turmer...",non vegetarian,-1,-1,spicy,main course,Assam,North East
243,Mishti Chholar Dal,"Chana dal, fresh coconut, ginger, cinnamon, ra...",vegetarian,10,30,sweet,main course,West Bengal,East
247,Prawn malai curry,"Coconut milk, prawns, garlic, turmeric, sugar",non vegetarian,15,50,spicy,main course,West Bengal,East


In [45]:
recommend_dishes("Disgust", ["milk", "yogurt"], True, False)

Based on your mood and preferences, I recommend trying the following dishes: 
Pinaca from the Goa state of the West region, which is a dessert dish.
Pinaca is a sweet dish and is vegetarian.
Pinaca contains the ingredients Brown rice, fennel seeds, grated coconut, black pepper, ginger powder.
Aloo methi from the Punjab state of the North region, which is a main course dish.
Aloo methi is a bitter dish and is vegetarian.
Aloo methi contains the ingredients Potato, fenugreek leaves, chillies, salt, oil.
Dudhi halwa from the Gujarat state of the West region, which is a dessert dish.
Dudhi halwa is a sweet dish and is vegetarian.
Dudhi halwa contains the ingredients Bottle gourd, green, raisins, sugar, clarified butter.
  Raisins is a fruit which is a part of the Mediterranean diet. The Mediterranean diet is associated with lower levels of depression as it causes less inflammation, less oxidative stress and improved vascular function.
